# Pull data from the WB data API and stores it into a local csv file for further processing

In [1]:
import os
import pandas as pd
import numpy as np
from pandas_datareader import wb
%load_ext autoreload
%autoreload 2

In [2]:
# WB country name translation
any_name_to_wb_name = pd.read_csv("../git/country_names/out/any_name_to_wb_name.csv",index_col="any",
                                  squeeze=True, encoding="utf-8")

In [3]:
cwd = os.getcwd()
os.chdir("../git/wb_api_wrapper/")
from  wb_api_wrapper import * #Import functions to obtain World Bank data from the web and code to rename country names
os.chdir(cwd) #Get back to initial working directory

## Now we start pulling WB data off the internet using the web api wrapper

## Look for the income share of the  bottom quintile

In [4]:
search_wb("income share") # Search for World Bank data sets which have a description containing "income share"

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:258: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  data = data.sort(columns='id')


,id,name,source
137,3.0.IncShr.q1,Income Share of First Quintile,LAC Equity Lab
138,3.0.IncShr.q2,Income Share of Second Quintile,LAC Equity Lab
139,3.0.IncShr.q3,Income Share of Third Quintile,LAC Equity Lab
140,3.0.IncShr.q4,Income Share of Fourth Quintile,LAC Equity Lab
141,3.0.IncShr.q5,Income Share of Fifth Quintile,LAC Equity Lab
9881,SI.DST.02ND.20,Income share held by second 20%,World Development Indicators
9882,SI.DST.03RD.20,Income share held by third 20%,World Development Indicators
9883,SI.DST.04TH.20,Income share held by fourth 20%,World Development Indicators
9884,SI.DST.05TH.20,Income share held by highest 20%,World Development Indicators
9885,SI.DST.10TH.10,Income share held by highest 10%,World Development Indicators


Pull the data using get_wb_df

In [5]:
wb_data_inc20 = get_wb_df("SI.DST.FRST.20", "income bottom quintile") # This pulls the income share of the bottom quintile

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


In [6]:
wb_data_inc20 = mrv(wb_data_inc20,include_year=True)

In [7]:
wb_data_inc20['value'] = wb_data_inc20['value']/100

In [8]:
wb_data_inc20.head(3)

,value,year
country,,
Albania,0.0885,2012
Algeria,0.0696,1995
Angola,0.0543,2008


In [9]:
wb_data_inc20.rename(columns={'value':'Income share of bootom 20% [SI.DST.FRST.20]'}, inplace=True)

## Now look for financial savings data from FINDEX

In [10]:
# If you want to see more outpit rows/cols/content, add one of the below:"
# with pd.option_context('display.max_rows', 999, 'display.max_columns', 3):
# pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth',200) # This command increases the maximum width of columns. It can interfer with Jupyter's ability to display all columns! It can be reste with: pd.reset_option('max_colwidth')
#pd.reset_option('max_colwidth')

In [35]:
# The difference between the [w1] and [w2] and [ts] datasets are the availability of data: w1 contains 2011 data,
# w2 2014 data and ts both years
wb.search('.*saved at a financial');

For savings at financial institutions, there are several indicators available which could all be interesting:
10989 	WP11648.1 	Saved at a financial institution in the past year (% age 15+)
10997 	WP11648.8 	Saved at a financial institution in the past year, income quintiles I (lowest) and II (% age 15+)
11521 	WP15165.1 	Saved at a financial institution (% age 15+) [w2]
11529 	WP15165.8 	Saved at a financial institution, income, poorest 40% (% ages 15+) [w2]
11667 	WP_time_04.8 	Saved at a financial institution, income, poorest 40% (% ages 15+) [ts]


In [12]:
#savings_data = wb.download(indicator=['WP11648.1','WP11648.8','WP15165.1','WP15165.8','WP_time_04.8'], country='all', start=1990, end=2015)
#savings_data_p40_w2 = get_wb_df('WP15165.8', 'Saved, poorest, w2')
savings_data_p40 = get_wb_df('WP_time_04.8', 'Saved, poorest, ts')
savings_data_r60 = get_wb_df('WP_time_04.9', 'Saved, poorest')

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


In [13]:
savings_data_p40.drop('MRV', inplace='True', level=1) #Drop the 'MRV' entry, as it messes up the further processing
savings_data_r60.drop('MRV', inplace='True', level=1) #Drop the 'MRV' entry, as it messes up the further processing
savings_data_p40 = mrv(savings_data_p40, include_year=True)
savings_data_r60 = mrv(savings_data_r60, include_year=True)
savings_data_p40['value'] = savings_data_p40['value']/100
savings_data_r60['value'] = savings_data_r60['value']/100

In [14]:
savings_data_r60.head(2)

,value,year
country,,
Afghanistan,0.044609,2014
Albania,0.096454,2014


In [15]:
savings_data_p40.rename(columns={'value':'Saved at a financial institution, 40% poorest [WP_time_04.8]'}, inplace=True)
savings_data_r60.rename(columns={'value':'Saved at a financial institution, 60% richest [WP_time_04.9]'}, inplace=True)

In [16]:
savings_data_p40.head(2)

,"Saved at a financial institution, 40% poorest [WP_time_04.8]",year
country,,
Afghanistan,0.022336,2014
Albania,0.044618,2014


In [17]:
savings_data_r60.head(2)

,"Saved at a financial institution, 60% richest [WP_time_04.9]",year
country,,
Afghanistan,0.044609,2014
Albania,0.096454,2014


## World Bank WGI Good governance indicators

In [18]:
wb.search('Political Stability and Absence of Violence/Terrorism');

In [19]:
PV = get_wb_mrv('PV.EST', 'PV_EST')
PV_PER_RNK = get_wb_mrv('PV.PER.RNK','PV_PER_RNK')
PV_PER_RNK_UPPER = get_wb_mrv('PV.PER.RNK.UPPER','PV_PER_RNK_UPPER')
PV_STD_ERR = get_wb_mrv('PV.STD.ERR','PV_STD_ERR')
PV = pd.concat([PV,PV_PER_RNK, PV_PER_RNK_UPPER, PV_STD_ERR], axis=1)

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


In [20]:
PV.head(4)

,PV_EST,PV_PER_RNK,PV_PER_RNK_UPPER,PV_STD_ERR
country,,,,
Afghanistan,-2.458067,2.912621,5.797101,0.221863
Albania,0.473255,61.650486,75.362320,0.204799
Algeria,-1.171418,10.194175,18.840580,0.209125
American Samoa,1.126468,89.320389,99.516907,0.320726


In [21]:
wb.search('Government Effectiveness')

,id,name,source,sourceNote,sourceOrganization,topics
5062,GE.EST,Government Effectiveness: Estimate,Worldwide Governance Indicators,"Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy...","b'Worldwide Governance Indicators, The World Bank'",
5063,GE.NO.SRC,Government Effectiveness: Number of Sources,Worldwide Governance Indicators,"Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy...","b'Worldwide Governance Indicators, The World Bank'",
5064,GE.PER.RNK,Government Effectiveness: Percentile Rank,Worldwide Governance Indicators,"Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy...","b'Worldwide Governance Indicators, The World Bank'",
5065,GE.PER.RNK.LOWER,"Government Effectiveness: Percentile Rank, Lower Bound of 90% Confidence Interval",Worldwide Governance Indicators,"Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy...","b'Worldwide Governance Indicators, The World Bank'",
5066,GE.PER.RNK.UPPER,"Government Effectiveness: Percentile Rank, Upper Bound of 90% Confidence Interval",Worldwide Governance Indicators,"Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy...","b'Worldwide Governance Indicators, The World Bank'",
5067,GE.STD.ERR,Government Effectiveness: Standard Error,Worldwide Governance Indicators,"Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy...","b'Worldwide Governance Indicators, The World Bank'",
5183,GV.GOVT.EF.ES,Government Effectiveness (estimate),Africa Development Indicators,"Government effectiveness measures the quality of public services, the quality and degree of independence from political pressures of the civil service, the quality of policy formulation and implem...",b'World Bank Institute.',
5184,GV.GOVT.EF.NO,Government Effectiveness (number of surveys/polls),Africa Development Indicators,See definition GV.GOVT.EF.ES. This is the number of surveys/polls used to derive the GV.GOVT.EF.ES.,b'World Bank Institute.',
5185,GV.GOVT.EF.SE,Government Effectiveness (standard error),Africa Development Indicators,"See definition GV.GOVT.EF.ES. Inherent to all Governance Indicators is a margin of error, which might vary from country to country, normally attributable to two factors: (i) cross-country differen...",b'World Bank Institute.',


In [22]:
GE = get_wb_mrv('GE.EST', 'GE_EST')
GE_PER_RNK = get_wb_mrv('GE.PER.RNK','GE_PER_RNK')
GE_PER_RNK_UPPER = get_wb_mrv('GE.PER.RNK.UPPER','GE_PER_RNK_UPPER')
GE_STD_ERR = get_wb_mrv('GE.STD.ERR','GE_STD_ERR')
GE = pd.concat([GE,GE_PER_RNK, GE_PER_RNK_UPPER, GE_STD_ERR], axis=1)

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


### Get (locally saved) data from corruption. Take the TI Corruption Index

In [23]:
CPI = pd.read_excel('Datasets/Corruption/Data & methodology/Data/CPI 2015_data.xlsx', index_col=2)
CPI.index.rename('country', inplace=True)
CPI.sort_index(inplace=True)
CPI.drop(['Region', 'Rank2', 'Country2', 'CPI2015(2)'], axis=1, inplace=True)
CPI.head(2)

,Rank,CPI2015,wbcode,World Bank CPIA,World Economic Forum EOS,Bertelsmann Foundation TI,African Dev Bank,IMD World Competitiveness Yearbook,Bertelsmann Foundation SGI,World Justice Project ROL,...,IHS Global Insight,PERC Asia Risk Guide,Freedom House NIT,Number of Sources,Std Deviation of Sources,Standard Error,Minimum,Maximum,Lower CI,Upper CI
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,166,11,AFG,12.0,NaN,19.0,NaN,NaN,NaN,2.0,...,11.0,NaN,NaN,4,6.98,3.49,2,19,5.2415,16.7585
Albania,88,36,ALB,NaN,49.0,36.0,NaN,NaN,NaN,18.0,...,32.0,NaN,37.0,7,9.48,3.58,18,49,30.0930,41.9070


## GINI Indicator

In [24]:
wb.search('GINI');

In [25]:
gini = get_wb_df('SI.POV.GINI', 'SI.POV.GINI - Gini Index')
gini = pd.DataFrame(mrv(gini))
gini= gini.rename(index=any_name_to_wb_name.to_dict())

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


## Remittances

In [26]:
wb.search('remittances');

In [27]:
remittances_received_gdp = get_wb_df('BX.TRF.PWKR.DT.GD.ZS','BX.TRF.PWKR.DT.GD.ZS - pers. remittances received, % GDP')
remittances_received_gdp = pd.DataFrame(mrv(remittances_received_gdp)).rename(index=any_name_to_wb_name)

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


In [28]:
remittances_received_usd = get_wb_df('BX.TRF.PWKR.CD.DT','BX.TRF.PWKR.CD.DT - pers. remittances received, curr. US$')
remittances_received_usd = pd.DataFrame(mrv(remittances_received_usd)).rename(index=any_name_to_wb_name)

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


In [29]:
remittances = pd.concat([remittances_received_usd, remittances_received_gdp], axis=1)

In [30]:
remittances.index.rename('country', inplace=True)

## GDP

In [31]:
gdp = wb.search('GDP')
gdp.query('name=="GDP, PPP (current international $)"')

,id,name,source,sourceNote,sourceOrganization,topics
7455,NY.GDP.MKTP.PP.CD,"GDP, PPP (current international $)",World Development Indicators,PPP GDP is gross domestic product converted to international dollars using purchasing power parity rates. An international dollar has the same purchasing power over GDP as the U.S. dollar has in t...,"b'World Bank, International Comparison Program database.'",Economy & Growth


In [32]:
gdp = get_wb_df('NY.GDP.MKTP.PP.CD','GDP per cap, PPP')
gdp = mrv(gdp)
gdp = pd.DataFrame(gdp).rename(index=any_name_to_wb_name)

/Users/steffen/anaconda/lib/python3.5/site-packages/pandas_datareader/wb.py:159: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  out = out.convert_objects(convert_numeric=True)


## Regions

In [33]:
country_classification = wb.get_countries()
country_classification = country_classification[['name', 'iso3c', 'incomeLevel', 'region']]

## Country income groups

## Save everything to csv files

In [34]:
remittances.to_csv('Datasets/WDI remittances.csv', encoding='utf-8')
gdp.to_csv('Datasets/GDP_pc_PPP.csv', encoding='utf-8')
gini.to_csv('Datasets/WDI GINI.csv', encoding='utf-8')
CPI.to_csv('Datasets/CPI_2015.csv',  encoding='utf-8')
PV.to_csv('Datasets/WGI Political Stability and Violence.csv',  encoding='utf-8')
GE.to_csv('Datasets/WGI Government Effectiveness.csv',  encoding='utf-8')
savings_data_p40.to_csv('Datasets/FINDEX saved_p40.csv',  encoding='utf-8')
savings_data_r60.to_csv('Datasets/FINDEX saved_r60.csv',  encoding='utf-8')
wb_data_inc20.to_csv('Datasets/WDI SI.DST.FRST.20.csv', encoding='utf-8')
country_classification.to_csv('Datasets/WB country_categories.csv', encoding='utf-8')